# 第4回講義 宿題

## 課題

CNNを用いて、FashionMNISTの高精度な分類器を実装してみましょう。
モデルのレイヤーを変更してみるなどして精度の向上にチャンレンジして下さい。 精度上位者はリーダーボードに載ります。

### 目標値

Accuracy 93%


### ルール

- 訓練データはx_train、 t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください。
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください。**
- ただし、**torch.nn.Conv2dのような高レベルのAPIは使用しないで下さい。**具体的には、nn.Parameter, nn.Module, nn.Sequential, nn.functional以外のnn系のAPIです。
- torchvision等で既に実装されているモデルも使用しないで下さい。

### 提出方法

- 2つのファイルを提出していただきます。
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください。
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)。

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。
- 定時に採点しLeader Boardを更新します。(採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします。

### データの読み込み

- この部分は修正しないでください

In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import inspect

nn_except = ["Module", "Parameter", "Sequential", "functional"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0]) 

"""
#学習データ
x_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210506/data/x_train.npy')
t_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210506/data/t_train.npy')
    
#テストデータ
x_test = np.load('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210506/data/x_test.npy')
"""
#学習データ
x_train = np.load('/Users/明朗/workspace/deeplearningUT/Lecture04/data/x_train.npy')
t_train = np.load('/Users/明朗/workspace/deeplearningUT/Lecture04/data/t_train.npy')
    
#テストデータ
x_test = np.load('/Users/明朗/workspace/deeplearningUT/Lecture04/data/x_test.npy')


class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 1, 28, 28).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 1, 28, 28).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 畳み込みニューラルネットワーク(CNN)の実装

In [7]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [36]:
rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Conv(nn.Module):
    # WRITE ME
    def __init__(self, filter_shape, function=lambda x: x, stride=(1, 1), padding=0):
        super().__init__()
        # Heの初期値
        fan_in = filter_shape[1] * filter_shape[2] * filter_shape[3]
        # filter_shape: (出力チャンネル数)x(入力チャンネル数)x(縦の次元数)x(横の次元数)
        fan_out = filter_shape[0] * filter_shape[2] * filter_shape[3]

        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/fan_in),
                        np.sqrt(6/fan_in),
                        size=filter_shape
                    ).astype('float32')))

        # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.b = nn.Parameter(torch.tensor(np.zeros((filter_shape[0]), dtype='float32')))
        self.function = function
        self.stride = stride
        self.padding = padding
        
    def forward(self, x):
        u = F.conv2d(x, self.W, bias=self.b, stride=self.stride, padding=self.padding)
        return self.function(u)


class Pooling(nn.Module):
    # WRITE ME
    def __init__(self, ksize=(2, 2), stride=(2, 2), padding=0):
        super().__init__()
        self.ksize = ksize
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        return F.max_pool2d(x, kernel_size=self.ksize, stride=self.stride, padding=self.padding)


class Flatten(nn.Module):
    # WRITE ME
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.size()[0], -1)


class Dense(nn.Module):
    # WRITE ME
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        # He Initialization
        # in_dim: 入力の次元数、out_dim: 出力の次元数              
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/in_dim),
                        np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))

        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function
    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)


conv_net = nn.Sequential(
    Conv((64, 1, 3, 3), F.relu, padding=1),                  #28,28,1 -> 28,28,64
    Pooling((2, 2)),                                         #28,28,64 -> 14,14,64
    Conv((128, 64, 3, 3), F.relu, padding=1),                #14,14,64 -> 14,14,128
    Pooling((2,2), stride=(1,1)),
    Conv((256, 128, 3, 3), F.relu, padding=1),               #14,14,128 -> 14,14,256
    Pooling((2, 2), stride=(1,1)),
    Conv((256, 256, 3, 3), F.relu, padding=1),               #14,14,256 -> 14,14,256
    Pooling((2, 2)),                                         #14,14,256 -> 7,7,256
    Conv((512, 256, 3, 3), F.relu, padding=1),               #7,7,256 -> 7,7,512
    Pooling((2, 2)),                                         #7,7,512 -> 3,3,512
    Flatten(),
    Dense(3*3*512, 10)
)# WRITE ME
"""
    Conv((194, 128, 3, 3), F.relu, padding=1),               #3,3,128 -> 3,3,194
    Pooling((2, 2)),
    Conv((384, 194, 3, 3), F.relu, padding=1),  
    Pooling((2, 2)),
    Conv((256, 384, 3, 3), F.relu, padding=1),  
    Pooling((2, 2)),  
"""  

n_epochs = 50
lr = 0.01

print(device)
conv_net.to(device)
optimizer = optim.SGD(conv_net.parameters(), lr=lr) # WRITE ME

cuda


In [37]:
for xx, tt in dataloader_train:
    break
print(xx.shape)
print(tt)

torch.Size([32, 1, 28, 28])
tensor([0, 3, 5, 6, 5, 6, 9, 9, 9, 2, 7, 9, 9, 4, 3, 7, 3, 7, 6, 3, 7, 9, 8, 4,
        3, 2, 0, 2, 6, 9, 4, 5])


In [38]:
import tqdm
for epoch in tqdm.tqdm(range(n_epochs)):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    conv_net.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        # WRITE ME
        conv_net.zero_grad()
        x = x.to(device)
        t_hot = torch.eye(10)[t] #10??
        t_hot = t_hot.to(device)
        y = conv_net.forward(x)
        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()
        loss.backward()
        optimizer.step()
        pred = y.argmax(1)

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    conv_net.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        # WRITE ME
        conv_net.zero_grad()
        x = x.to(device)
        t_hot = torch.eye(10)[t] #10??
        t_hot = t_hot.to(device)
        y = conv_net.forward(x)
        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()
        pred = y.argmax(1)

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()
    if (epoch+1)%5==0:
        print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
            epoch,
            np.mean(losses_train),
            train_true_num/train_num,
            np.mean(losses_valid),
            valid_true_num/valid_num
        ))

100%|██████████| 50/50 [16:12<00:00, 19.45s/it]EPOCH: 49, Train [Loss: 0.001, Accuracy: 1.000], Valid [Loss: 0.395, Accuracy: 0.928]



In [39]:
conv_net.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = conv_net.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
"""
submission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210506/submission_pred.csv', header=True, index_label='id')
"""
import os
os.makedirs('output', exist_ok=True)
submission.to_csv('output/submission_pred_3_%d'%n_epochs+'.csv', header=True, index_label='id')